In [1]:
%pwd 

'/workspaces/OpenDVCW/train_models'

In [2]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import OpenDVCW
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

/workspaces/OpenDVCW


In [3]:
BATCH_SIZE = 1
EPOCHS = 700
STEPS_PER_EPOCH = 200
Height = 240
Width = 240
Channel = 3
lmbda = 1024
lr_init = 1e-4
early_stop = 15
I_QP=27

args = OpenDVCW.Arguments()
last = 0
checkponts_last_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last, Width, Height)
checkponts_new_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last+1, Width, Height)
save_name = "model_save_wavelets_L_{}_{}_{}x{}".format(lmbda, last+1, Width, Height)


In [4]:
%load_ext tensorboard


In [5]:
rm -rf ./logs/

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [7]:
model = OpenDVCW.OpenDVC(width=Width, height=Height, batch_size=BATCH_SIZE, num_filters=128, lmbda=lmbda)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("folder_cloud_test.npy", 
                                    BATCH_SIZE,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

# print("Loading weights")
# model.load_weights(checkponts_last_path)


2022-04-05 04:59:21.246492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 04:59:21.297814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 04:59:21.298643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 04:59:21.306268: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

* [Model compiled]...
* [Loading dataset]...


In [8]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [9]:
for layer in model.layers:
    print(layer.name, layer.trainable)

mv_analysis True
mv_synthesis True
res_analysis True
res_synthesis True
wavelets_optical_flow True
motion_compensation True


In [10]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE,
                callbacks=[
                    Callbacks.MemoryCallback(),
                    Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

Epoch 1/700


2022-04-05 05:00:00.235806: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-04-05 05:00:43.603944: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


  6/200 [..............................] - ETA: 56s - loss: 62.1167 - bpp: 5.3752 - mse: 0.0554WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2777s vs `on_train_batch_end` time: 0.2894s). Check your callbacks.


200/200 [==============================] - ETA: 0s - loss: 16.3186 - bpp: 5.3043 - mse: 0.0108[MemoryCallback]:  4838424

Epoch 1: loss improved from inf to 16.31858, saving model to checkpoints_wavelets_L_1024_1_240x240/
200/200 [==============================] - 119s 265ms/step - loss: 16.3186 - bpp: 5.3043 - mse: 0.0108
Epoch 2/700
200/200 [==============================] - ETA: 0s - loss: 7.8212 - bpp: 5.1647 - mse: 0.0026[MemoryCallback]:  4935084

Epoch 2: loss improved from 16.31858 to 7.82117, saving model to checkpoints_wavelets_L_1024_1_240x240/
200/200 [==============================] - 56s 276ms/step - loss: 7.8212 - bpp: 5.1647 - mse: 0.0026
Epoch 3/700
200/200 [==============================] - ETA: 0s - loss: 7.1008 - bpp: 5.0292 - mse: 0.0020[MemoryCallback]:  5027096

Epoch 3: loss improved from 7.82117 to 7.10078, saving model to checkpoints_wavelets_L_1024_1_240x240/
200/200 [==============================] - 55s 270ms/step - loss: 7.1008 - bpp: 5.0292 - mse: 0.0020


In [11]:
path = load.load_random_path("folder_cloud_test.npy")
i=0
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00056/0723/im1.png
compress
in the compress
decompress
in decompress


In [12]:
model.save(save_name, save_format="tf")

in the compress
in decompress


INFO:tensorflow:Assets written to: model_save_wavelets_L_1024_1_240x240/assets


INFO:tensorflow:Assets written to: model_save_wavelets_L_1024_1_240x240/assets
